In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive') 

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

In [2]:

data_path = 'C:/Santosh/AI/K-Comp/Ion/Data/'
train_signal_file = data_path + 'train_signal.npy'
train_channels_file = data_path + 'train_channels.npy'
test_signal_file = data_path + 'test_signal.npy'

train_signal = np.load(train_signal_file)
train_channels = np.load(train_channels_file)
test_signal = np.load(test_signal_file)

valid_signal = train_signal[:,400000:]
valid_channels = train_channels[:,400000:]
train_signal = train_signal[:,:400000]
train_channels = train_channels[:,:400000]


In [10]:
for i in range(10):
    print(i,"   ",np.mean(train_signal[i]),"    ", np.mean(train_signal[i][:10000]))
    

0     -2.6775800939999996      -2.6967759799999995
1     -2.6871994944881026      -2.6056200899810142
2     -1.81228728525      -1.80535439
3     -0.09958405574999998      -0.12528418999999996
4     3.3522119407500006      3.30219928
5     1.6927178065000004      1.6928120299999998
6     -1.8023615880461925      -1.746254199981021
7     -0.10431139079619701      -0.28071283998104035
8     1.652365197953807      1.598213640018976
9     3.307611075203807      3.2610364800189746


In [13]:
for i in range(4):
    print(i,"   ", np.mean(test_signal[i][:100000]))
    print(i,"   ", np.mean(test_signal[i][100000:200000]))
    print(i,"   ", np.mean(test_signal[i][200000:300000]))
    print(i,"   ", np.mean(test_signal[i][300000:400000]))
    print(i,"   ", np.mean(test_signal[i][400000:]))

0     -2.6032884469524418
0     -0.08312186995240792
0     1.65359524
0     -2.5381632330000006
0     -1.8128117409524258
1     3.2824766689999993
1     1.649553205047574
1     3.3342728550475904
1     -2.6025783369524262
1     -0.06451003300000001
2     -2.614403360215041
2     -2.594898185065903
2     -2.614815458401149
2     -2.595055447502683
2     -2.5772216009523947
3     -2.6058661759999993
3     -2.6091033410000004
3     -2.6315788
3     -2.6564056670000005
3     -2.602071999


In [ ]:

def windowed_dataset(all_series, all_channels, window_size, batch_size, shuffle_buffer):
    
    all_dataset = np.zeros((end - start)*1*(window_size + 1))
    cnt = 0
    s = 0
    for series in all_series:
        
        for i in range(start,end):
            all_dataset[s:s + window_size] = series[i-window_size+1:i+1]
            s += window_size
            all_dataset[s] = all_channels[cnt][i]
            s+=1

        cnt+=1
        break
    #print(all_dataset)
    series = np.array(all_dataset)
    
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size + 1, shift=window_size+1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    dataset = dataset.shuffle(shuffle_buffer).map(lambda window: (window[:-1], window[-1]))
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset

dataset = windowed_dataset(train_signal, train_channels, window_size, batch_size, shuffle_buffer)

In [ ]:
y_true = valid_channels[0][window_size - 1:]
valid_input = np.zeros((100000 - window_size + 1 , window_size))
s = 0
for input in valid_input:
    input = valid_signal[s:window_size]
    s+=window_size
y_true = (np.array(y_true).astype(int))


In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),
                      input_shape=[None]),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
  tf.keras.layers.Dense(11,activation = 'softmax')
])

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])
history = model.fit(dataset, epochs=5, steps_per_epoch=100)

In [ ]:

window_size = 30
batch_size = 100
shuffle_buffer = 1000
start = window_size - 1
end = 400000


In [ ]:
batches = [0,1,2,6]
for k in batches:
    print(k)
    y_true = valid_channels[k][window_size - 1:]
    valid_input = np.zeros((100000 - window_size + 1 , window_size))
    s = 0
    for input in valid_input:
        input = valid_signal[s:window_size]
        s+=window_size

    y_pred = model.predict(valid_input)
    y_true = (np.array(y_true).astype(int))
    y_pred_1 = np.zeros(y_true.shape)
    for j in range(y_pred.shape[0]):
        maxval = 0
        index = 0
        for i in range(11):
            if(y_pred[j][i] > maxval):
                maxval = y_pred[j][i]
                index = i
        y_pred_1[j] = index
    
    print(y_true.shape)
    y_pred_1 = np.array(y_pred_1).astype(int)
    y_true = np.array(y_true).astype(int)
    #print(f1_score(y_true, y_pred_1, average='macro'))
    tp1 = 0
    tn1 = 0
    fp1 = 0
    fn1 = 0
    tp0 = 0
    tn0 = 0
    fp0 = 0
    fn0 = 0
    for i in range(y_true.shape[0]):

        if(y_true[i]==1 and y_pred_1[i]==1):
            tp1+=1
            tn0+=1
        if(y_true[i]==1 and y_pred_1[i]==0):
            fn1+=1
            fp0+=1
        if(y_true[i]==0 and y_pred_1[i]==1):
            fn0+=1
            fp1+=1
        if(y_true[i]==0 and y_pred_1[i]==0):
            tp0+=1
            tn1+=1
    p1 = tp1/(tp1+fp1)
    r1 = tp1/(tp1+fn1)
    p0 = tp0/(tp0+fp0)
    r0 = tp0/(tp0+fn0)
    pre = (p1+p0)/2
    rec = (r1+r0)/2
    f1 = 2*(pre*rec)/(pre+rec)
    print(f1)



